## makemore 5 - backporp ninja

- [Yes you should understand backprop](https://medium.com/@karpathy/yes-you-should-understand-backprop-e2f06eab496b)

refers to backprop as a leaky abstraction and says you should understand it.  I agree.  Need to gain an understanding of the internals 

- [Backpropagation calculus](http://neuralnetworksanddeeplearning.com/chap2.html)

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# read in all of the words from ./data/names.txt
words = open('./data/names.txt').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [6]:
# build the vocab of characters and mappings to/from integers

chars = sorted(list(set(''.join(words))))
stoi = {s:i + 1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [8]:
# build the dataset
block_size = 3 # max length of names in the dataset

def build_dataset(words):
    X, Y = [], []

    for w in words:
        context = [0] * block_size
        for ch in w + '.': 
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix] # crop and append
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random 
random.seed(42)
random.shuffle(words)
n1 = int(len(words) * 0.8)
n2 = int(len(words) * 0.9)

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])


torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [9]:
# ok boiler plate done... now time to get to business

In [10]:
# utility function we will use later when comparing manual gradients to PyTorch gradients

def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')
    

In [12]:
n_embd = 10 # the dimensionality of the character embedding vectors 
n_hidden = 64 # the number of neurons is the hidden layer

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd), generator=g) 

# layer1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3) / ((n_embd * block_size) ** 0.5)
b1 = torch.randn(n_hidden, generator=g) * 0.1
# layer 2 
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden)) * 0.1 + 1.0 
bnbias = torch.randn((1, n_hidden)) * 0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True 

4137


In [13]:
batch_size = 32
n = batch_size
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (n,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

In [14]:


# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors

# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation

# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias

# Non-linearity
h = torch.tanh(hpreact) # hidden layer

# Linear layer 2
logits = h @ W2 + b2 # output layer

# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss



tensor(3.3601, grad_fn=<NegBackward0>)

__the chain rule__

this comes up quite a bit as we work through all of the math of our backpropogation. I needed a refresher, and figure you probably will again. 

__prompt: you're an intelligent professor, explaining something technical to an audience with a high aptitude for learning new things, but is only moderately exposed to the subject:__
__what is the chain rule, in calculus, and how does it apply to computer science - specifically neural networks for ai__

Certainly, let's first break down the Chain Rule in calculus and then connect it with its significance in artificial intelligence, particularly neural networks.

The Chain Rule is a fundamental concept in calculus. It states that if you have a function that is composed of other functions, the derivative (rate of change) of the total function is the product of the derivatives of the component functions.

In formal mathematical language, let's say we have a function h(x) = g(f(x)), where g and f are differentiable functions. Then, the derivative of h with respect to x is given by h'(x) = g'(f(x)) * f'(x).

Here's an intuitive way to think about it. The Chain Rule essentially says that if we have some machinery (function) that first transforms x into another value via f(x), and then further transforms f(x) into another value via g(f(x)), the total change in the output as x changes is the product of the local changes.

Now, let's turn our attention to how this applies to computer science, specifically in the context of artificial intelligence and neural networks.

Neural networks can be thought of as a composition of functions, much like our function h(x) = g(f(x)) from before. In a neural network, the input data is transformed by the network's layers into an output. Each layer is like one of our component functions, which takes in data, performs a transformation (often a linear transformation followed by a non-linear activation function), and sends the result to the next layer.

The error of the network's predictions is calculated using a loss function, which is a measure of how far off the network's output is from the true output. We want to minimize this error, and to do that we use a process called backpropagation, which uses the gradient descent algorithm. This is where the Chain Rule comes into play.

Gradient descent uses the derivatives (gradients) of the loss function with respect to the network's weights to determine how to adjust those weights. The Chain Rule is used to calculate these gradients because the loss function is a composition of the functions represented by the network's layers. In essence, to know how the error changes as we change a particular weight, we have to 'chain together' the derivatives of all the transformations that happened between that weight's input and the final output.

In this way, the Chain Rule is a critical underpinning of the learning algorithm for neural networks. Without it, we wouldn't be able to effectively adjust the weights in our networks and they wouldn't be able to learn from their errors.

I hope this clarifies the concept and its application for you! Let me know if you have any further questions.

- This one helped a lot too...dxd​[f(g(x))]=f′(g(x))g′(x)
- The derivative of the entire loss function is the product of the derivatives of the individual layers/functions that make up the loss function. This is the chain rule in action.
    - [Khan Academy - Chain Rule](https://www.khanacademy.org/math/ap-calculus-ab/ab-differentiation-2-new/ab-3-1a/a/chain-rule-review)


In [ ]:
# backprob through the whole thing manually
# exactly all of the variables 
# as they are defined in the forward pass above, one by one

dlogprobs = torch.zeros_like(logprobs) 
dlogprobs[range(n), Yb] = -1.0 / n 
dprobs = (1.0/ probs) * dlogprobs 
dcounts_sum_inv = (counts * dprobs).sum(1, keepdims=True) 
dcounts = counts_sum_inv * dprobs 
